# План на сегодня
1. Как разбить текст на слова? 
2. Как посчитать количество слов?
3. Все ли слова нужны? Удаление стоп-слов.
4. Как определить часть речи слова. 

Инструменты: nltk, pymorphy2, pymystem3 

# Токенизация и подсчет количества слов


## Сколько слов в этом предложении?
* На дворе трава, на траве дрова, не руби дрова на траве двора.*


### Токен и тип

**  Тип **  – уникальное слово из текста

** Токен **  – тип и его позиция в тексте

** Лексема **  – абстрактная сущность слова


### Обозначения 
$N$ = число токенов

$V$ – словарь (все типы)

$|V|$ = количество типов в словаре (то есть число уникальных слов)

** Как связаны $N$ и $|V|$?**


### Закон Ципфа


**В любом достаточно большом тексте ранг типа обратно пропорционален его частоте: $f = \frac{a}{r}$ **

$f$ – частота типа, $r$  – ранг типа, $a$  – параметр, для славянских языков – около 0.07

Иными словами, если расмотреть список слов, упорядоченный по убыванию частоты их использования, то частота n-го слова в таком списке окажется приблизительно обратно пропорциональной его порядковому номеру n.


In [ ]:
from IPython.display import Image
Image("Zipf.png")

### Закон Хипса

**С увеличением длины текста (количества токенов), количество типов увеличивается в соответствии с законом: $|V| = K*N^b$ **


$N$  –  число токенов, $|V|$  – количество типов в словаре, $K, b$  –  параметры, обычно $K \in [10,100], b \in [0.4, 0.6]$

In [ ]:
Image("Hips.png")

# Проверим на практике!

## Анализ  сообщений vk.com

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
# Тексты из пабликов госслужб из ВК
df = pd.read_csv('vk_texts_with_sources.csv', usecols = ['text', 'source'])

df.text.dropna(inplace = True)
df.head()



### Что за данные?

Данные из госпабликов ВК. В поле source записан id группы.

In [ ]:
print(df.source.unique())
for group in df.source.unique():
    print('https://vk.com/' + group)

## Предварительный анализ коллекции

### Средняя длина текстов

In [ ]:
len_data = df.text.apply(len)
len_data.describe()

### Количество текстов из разных пабликов

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
counts = df.source.value_counts()
values = counts.tolist()
labels = counts.index.tolist()


y_pos = np.arange(len(labels))

 
plt.bar(y_pos, values, align='center', alpha=0.5)
plt.xticks(y_pos, range(len(labels)))

 
plt.show()

### Длины текстов (в символах)

In [ ]:
fig, ax = plt.subplots()

length = len_data[len_data < 10000].tolist()

n, bins, patches = ax.hist(length)

fig.show()

In [ ]:
print('Most typical length:', round(len_data.median()))

## Токенизация

Используем регулярные выражения, чтобы разбить тексты на слова

In [ ]:
import re
regex = re.compile("[А-Яа-я]+")

#Оставляем только слова, удаляем мусор.
def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""


df.text = df.text.str.lower()
df.text = df.text.apply(words_only)

df.text.iloc[0]

### Самые частые слова

In [ ]:
from nltk import FreqDist
n_types = []
n_tokens = []
tokens = []
fd = FreqDist()
for index, row in df.iterrows():
    tokens = row['text'].split()
    fd.update(tokens)
    n_types.append(len(fd))
    n_tokens.append(sum(fd.values()))
for i in fd.most_common(10):
    print(i)

** Вопрос на подумать **

Несут ли эти слова смысловую нагрузку? Много ли информации они содержат?

# Проверим законы на практике

### Закон Ципфа

In [ ]:
freqs = list(fd.values())
freqs = sorted(freqs, reverse = True)

fig, ax = plt.subplots()
ax.plot(freqs[:300], range(300))
plt.show()

** Вопрос на подумать ** Совпадает ли это с графиком? Выполняется ли закон Ципфа?

### Закон Хипса

In [ ]:
fig, ax = plt.subplots()
ax.plot(n_types, n_tokens)
plt.show()


** Вопрос на подумать ** Совпадает ли это с графиком? Выполняется ли закон Хипса?

** Вопрос на подумать ** Чему равно $b$?

In [ ]:
# Подбираем кривизну кривой
n_types = np.array(n_types)
b_ = ## 
plt.plot(n_types, n_types ** b_)

# Морфологический анализ

### Задачи морфологического анализа
* Разбор слова — определение нормальной формы  (леммы), основы (стема) и грамматических характеристик слова
* Синтез слова — генерация слова по заданным грамматическим характеристикам


### Морфологический процессор – инструмент морфологического анализа
* Морфологический словарь 
* Морфологический анализатор

### Лемматизация 
У каждого слова есть *лемма* (нормальная форма): 

* кошке, кошку, кошкам, кошкой $\implies$ кошка
* бежал, бежит, бегу $\implies$  бежать
* белому, белым, белыми $\implies$ белый

In [ ]:
sent1 = 'Действительно, на его лице не отражалось никаких чувств – ни проблеска сочувствия не было на нем, а ведь боль просто невыносима'
sent2 = 'У страха глаза велики .'

# Инструменты:

Два основных инструмента морфологического анализа:

** 1) библиотека pymorphy2 и анализатор MorphAnalyzer **

Представляет собой словарь вида.

    dict[word] = lemma 
    
Не умеет разрешать омонимию.
    
** 2) библиотека mystem и анализатор Mystem **

Разработка Яндекса.

Главное достоинство: умеет разрешать омонимию.

Про Mystem можно почитать в статье I. Segalovich (2003), A fast morphological algorithm with unknown word guessing induced by a dictionary for a web search engine:


https://cache-mskstoredata11.cdn.yandex.net/download.yandex.ru/company/iseg-las-vegas.pdf

In [ ]:
from pymorphy2 import MorphAnalyzer

m = MorphAnalyzer()
lemmas1 = [m.parse(word)[0].normal_form for word in sent1.split()]
print(' '.join(lemmas1))

In [ ]:
#mystem - запрещен использование проприетарная разраббез оплаты разботка Яндекса
#умеет разрешать омонимию (у страха глаза велики)
from pymystem3 import Mystem

m = Mystem()
lemmas2 = m.lemmatize(sent1)
print(''.join(lemmas2))

### Снятие омонимии

** Омонимия ** - совпадение слов разных по смыслу.

In [ ]:
m = MorphAnalyzer()
lemmas1 = [m.parse(word)[0].normal_form for word in sent2.split()]
print(' '.join(lemmas1))

m = Mystem()
lemmas2 = m.lemmatize(sent2)
print(''.join(lemmas2))

### Стемминг

** Основа слова ** - неизменяемая часть слова, которая выражает его лексическое значение.

** Стемминг ** - процесс нахождения лексической основы для заданного исходного слова.

$word = stem + affixes$

$affixes = preffixes + suffixes + interfixes + others$

В русском языке мы чаще всего хотим избавиться от суффиксов.




### Алгоритм Портера

* популярный алгоритм стемминга

* последовательно применяет ряд правил, отсекая суффиксы и окончания

* работает быстро, но допускает ошибки



### Ссылки:

Porter's homepage https://tartarus.org/martin/PorterStemmer/

Original paper https://www.emeraldinsight.com/doi/abs/10.1108/eb046814

In [ ]:
from nltk.stem.snowball import RussianStemmer

stemmer = RussianStemmer()
words = ['распределение', 'приставить', 'сделала', 'словообразование']
for w in words:
    stem = stemmer.stem(w)
    print(stem)

### 3 типа ошибок

** 1-ый вид ошибки ** 
Разные слова приводятся к одной основе.

In [ ]:
from nltk.stem.snowball import RussianStemmer

stemmer = RussianStemmer()
words = ['белый','белье']
for w in words:
    stem = stemmer.stem(w)
    print(stem)

** 2-ый вид ошибки ** 
Слова с одинаковыми основами приводятся к разным.

In [ ]:
from nltk.stem.snowball import RussianStemmer

stemmer = RussianStemmer()
words = ['трудный','трудность']
for w in words:
    stem = stemmer.stem(w)
    print(stem)

** 3-ый вид ошибки ** 
Не удаляется приставка.

In [ ]:
from nltk.stem.snowball import RussianStemmer

stemmer = RussianStemmer()
words = ['быстрый','побыстрее']
for w in words:
    stem = stemmer.stem(w)
    print(stem)

## Разбор слова 

** Граммема ** - грамматическое значение, понимаемое как один из элементов грамматической категории.

In [ ]:
word1 = 'семью'

In [ ]:
m = MorphAnalyzer()
parsed_word = m.parse(word1)
parsed_word

In [ ]:
m = Mystem()
parsed_word = m.analyze(word1)
parsed_word

### Что в имени твоем

In [ ]:
your_name = ### Enter your name here  
m = Mystem()
parsed_word = m.analyze(your_name)
parsed_word[0]

In [ ]:
name_parse = parsed_word[0]

grammema = name_parse['analysis'][0]['gr']
print(grammema)
if re.search('имя', grammema):
    print('It is really a name.')
    
if re.search('жен', grammema):
    print('You are female.')
elif re.search('муж', grammema):
    print('You are male.')
else:
    print('Omm... Could not recognise you...')

### Пришло время писать свой код!

### Задание

Найдите в списке персонажей романа "Война и мир"  (task.txt) все уникальные  женские имена.

Загружаем данные.

In [ ]:
text_file = open("task.txt", "r", encoding='utf-8')
lines = text_file.readlines()

print(len(lines))
for line in lines[1:50]:
    print(line)
text_file.close()

In [ ]:
### YOUR CODE HERE




## Первичная обработка текстов

### Еще раз загрузим те же самые данные

In [ ]:
df = pd.read_csv('vk_texts_with_sources.csv', usecols = ['text', 'source'])
df.text.dropna(inplace = True)

#WARNING! Если вы используете WINDOWS, то Mystem() может работать медленно!
#Если не хотите долго ждать, оставьте лишь часть данных!

#Раскомментить данную строчку для пользователей Windows
# df = df.head(100)

df.head()

### Удаление стоп-слов

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words('russian'))

In [ ]:
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д']
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""
 

In [ ]:
m = Mystem()

def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [ ]:
mystoplemmas = ['который','прошлый','сей', 'свой', 'наш', 'мочь']
def  remove_stoplemmas(text, mystoplemmas = mystoplemmas):
    try:
        return " ".join([token for token in text.split() if not token in mystoplemmas])
    except:
        return ""


In [ ]:
df.text = df.text.apply(remove_stopwords) 
df.text = df.text.apply(lemmatize)
df.text = df.text.apply(remove_stoplemmas) 

In [ ]:
lemmata = []
for index, row in df.iterrows():
    lemmata += row['text'].split()
fd = FreqDist(lemmata)
for i in fd.most_common(10):
    print(i)

## Бонус. Демо Natasha

Natasha - библиотека для поиска и извлечения именованных сущностей (Named-entity recognition) из текстов на русском языке. На данный момент разбираются упоминания персон, даты и суммы денег.

In [ ]:
from natasha import *


text = 'Экс-президента Франции Николя Саркози задержали по делу о финансировании его избирательной кампании 2007 года, \
передает французская газета Monde. Сейчас, по данным издания, политик находится в помещениях судебной полиции в Нантере. \
Бывший президент Франции пробудет под стражей минимум 48 часов. \
Весной 2012 года французское издание Mediapart опубликовало документы, в которых говорилось о передаче ливийским режимом \
50 млн евро на нужды президентской кампании Саркози. Судебные разбирательства по этому делу продолжаются во Франции до сих пор.'

#text = 'Под Нижним Новгородом передали Нижнему Новгороду передали 100 евро'

extractors = [NamesExtractor(), PersonExtractor(), DatesExtractor(), LocationExtractor(), OrganisationExtractor(), MoneyExtractor()]
for extractor in extractors:
    matches = extractor(text)
    for match in matches:
        print(match.span, match.fact)

Наташа выделяет именованные сущности на основе правил, в связи с чем она допускает ошибки.

### Пример неправильного разбора

In [ ]:
from natasha import *

# Отрывок статьи из Википедии о Хараки Мураками

text = 'Харуки Мураками (12 января 1949 года, Киото) — японский писатель и переводчик. \
        Его книги переведены на 50 языков и являются бестселлерами как в Японии, \
        так и за пределами его родной страны. '

#text = 'Под Нижним Новгородом передали Нижнему Новгороду передали 100 евро'

extractors = [NamesExtractor(), PersonExtractor(), DatesExtractor(), LocationExtractor(), OrganisationExtractor(), MoneyExtractor()]
for extractor in extractors:
    matches = extractor(text)
    for match in matches:
        print(match.span, match.fact)